In [37]:
from osgeo import gdal, osr, ogr
import matplotlib.pyplot as plt
import numpy as np
import os
import osmnx as ox
import rasterio
from rasterstats import zonal_stats
from rasterio.plot import show
import geopandas as gpd
from shapely.geometry import Polygon
import json

In [45]:
# CONSTANTS

# Base Coordinate System EPSG code 
BASE_CRS = 4326 

# Location center [minx, miny, maxx, maxy]
# LOC = [-19.3758794, 47.33124417, -19.07561176, 47.63146114]
# LOC = [-19.2758794, 47.43124417, -19.17561176, 47.53146114]
LOCS = [[-96, 31, -82, 39], [55, 2.38, 101, 38]]

# Raster directory
DIR = 'layers/'

# Which sampling statistic to output
STATS=['mean']

In [46]:
output = {}
output["Locations"] = LOCS

for ind, LOC in enumerate(LOCS):
    
    # Initialize empty holders
    tifs = []
    shps = []
    loc_output = {}

    # Add constants to location output file
    loc_output["Location"] = LOC
    loc_output["Stats"] = STATS
    loc_output["Bands"] = {}

    # Iterate through directory
    for subdir, dirs, files in os.walk(DIR):
        for f in files:
            path = subdir + os.sep + f
            if path.endswith(".tif"):
                tifs.append(path)
            elif path.endswith(".shp"):
                shps.append(path)

    for ds_path in tifs:
        # Select raster image 
        print("\nOpening " + ds_path)
        ds = gdal.Open(ds_path)

        # Get the number of layers
        print("Number of Layers: " + str(ds.RasterCount))
#         print(dir(ds.GetRasterBand(0).GetStatistics(True, True))

        # Get source coordinate reference type 
        projection = int(osr.SpatialReference(wkt=ds.GetProjection()).GetAttrValue('AUTHORITY',1))
        print("EPSG: " + str(projection))
        source = osr.SpatialReference()
        source.ImportFromEPSG(projection)

        # Set target coordinate reference 
        target = osr.SpatialReference()
        target.ImportFromEPSG(BASE_CRS)

        # Create point transformation
        transform = osr.CoordinateTransformation(source, target)

        # Get raster coordinates (rc) in source reference 
        rc = getRasterCoords(ds)
#         print(rc)

        # Convert raster coordinates to target reference
        rc_conv = np.array([])
        point = ogr.Geometry(ogr.wkbPoint)
        point.AddPoint(rc[0], rc[1])
        point.Transform(transform)
        rc_conv = np.concatenate((rc_conv, [point.GetX(), point.GetY()]))
        point.AddPoint(rc[2], rc[3])
        point.Transform(transform)
        rc_conv = np.concatenate((rc_conv, [point.GetX(), point.GetY()]))
#         print("Raster coordinates in target reference: " + str(rc_conv))

        # Check if raster contains the desired location
        if rasterContainsLoc(rc_conv):

            print("Processing file " + os.path.basename(ds_path) + "...", end=" ")
            for band in range(1, ds.RasterCount+1):
                model = rasterio.open(ds_path)
                crs = {'init': 'epsg:'+str(BASE_CRS)}
                polygon = gpd.GeoDataFrame(crs=crs, geometry=[returnPolygon(LOC)])
                place = polygon.to_crs(crs=model.crs)

                # Optional - Use location name as Polygon
#                 place = ox.geocode_to_gdf("California")
#                 place = ox.geocode_to_gdf(polygon)
#                 place = place.to_crs(crs=model.crs)

                # Optional - Visualize band 
#                 ax = place.plot(facecolor='None', edgecolor='red', linewidth=2)
#                 show((model, band), ax=ax)

                # Compute zonal stats 
                array = model.read(band)
                affine = model.transform
                zs = zonal_stats(place, array, affine=affine, stats=STATS)  

                # Write results to output dict
#                 if os.path.basename(ds_path) in loc_output["Bands"]:
                loc_output["Bands"][os.path.basename(ds_path) + "-" + str(band)] = zs[0]

            # Finished processing file bands
            print("Complete")

        # If raster does not contain desired location
        else:
            print("ERROR: Raster does not contain location: " + str(LOC) + ". Will not be included in results output.")

    output[ind] = loc_output
    
print("\nWriting location results to results.json...", end=" ")
with open('result.json', 'w') as fp:
    json.dump(output, fp, indent=2)
print("Complete")



Opening layers//img.tif
Number of Layers: 20
EPSG: 3857
ERROR: Raster does not contain location: [-96, 31, -82, 39]. Will not be included in results output.

Opening layers//basic.tif
Number of Layers: 28
EPSG: 4326
Processing file basic.tif... Complete

Opening layers//img.tif
Number of Layers: 20
EPSG: 3857
ERROR: Raster does not contain location: [55, 2.38, 101, 38]. Will not be included in results output.

Opening layers//basic.tif
Number of Layers: 28
EPSG: 4326
Processing file basic.tif... Complete

Writing location results to results.json... Complete


In [4]:
def getRasterCoords(ds):
    
    gt = ds.GetGeoTransform()
    w = ds.RasterXSize
    h = ds.RasterYSize
    
    minx = gt[0]
    maxy = gt[3] 
    miny = gt[3] + w*gt[4] + h*gt[5] 
    maxx = gt[0] + w*gt[1] + h*gt[2]
    
    return[minx, miny, maxx, maxy]

In [5]:
def rasterContainsLoc(rc):
    if LOC[0] >= rc[0] and LOC[3] <= rc[3] and LOC[1] >= rc[1] and LOC[3] <= rc[3]:
        return True
    else:
        return False

In [6]:
def returnPolygon(bounds):
    poly = Polygon([(LOC[0], LOC[1]), (LOC[2], LOC[1]), (LOC[2], LOC[3]), (LOC[0], LOC[3]), (LOC[0], LOC[1])])
    return(poly)